In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## Calls and Functions for reading data downloaded from SEC website 
header = "SETTLEMENT DATE|CUSIP|SYMBOL|QUANTITY (FAILS)|DESCRIPTION|PRICE"

path = r'C:\Users\watso\Desktop\TherinFintech\GME_Shenanigans\SEC_Files_CSV' # USE YOUR PATH 
all_files = glob.glob(path + "/*.csv")

GME_CUSIP_number = "36467W109"   
GME_symbol = 'GME'

CUSIP_number = "36467W109"   # Default  
symbol = 'GME'               # Default

# Main function - passing the CUSIP number (most important) and setting the symbol will deliver
#                 FTD data sorted by CUSIP number 
# All other functions here exist to support this 
def return_dataframe(cusip_number,symbol):
    df = read_ftd_data_any_stock(cusip_number,symbol)
    df = fix_dataframe(df)
    return df

# Use return_dataframe as your function to call FTD Data 


def read_ftd_file(csv_path, cusip_number, symbol):
    data = pd.read_csv(
    Path(csv_path),
    index_col=False
    )
    
    data[header] = data[header].str.replace('|',',')
    new_data = data.rename(columns={header:'SETTLEMENT_DATE,CUSIP,SYMBOL,QUANTITY_FAILS,DESCRIPTION,PRICE'})
    Header = "Header"
    new_data = data.rename(columns={header:Header})
    
    new_data = new_data.Header.str.split(",",expand=True)
    new_data = new_data.rename(columns={0:'Date',1:'CUSIP',2:'SYMBOL',3:'QUANTITY_FAILS',4:'DESCRIPTION',5:'PRICE'})
    
    new_data = new_data.set_index("CUSIP")
    new_data = new_data.loc[cusip_number]
    
    new_data = new_data[['Date','QUANTITY_FAILS','PRICE']]
    new_data.rename(columns={'PRICE':symbol},inplace=True)
    new_data.set_index('Date',inplace=True)  
    
    new_data = new_data.reset_index()
    new_data['Date'] = pd.to_datetime(new_data['Date'])
    new_data.set_index('Date',inplace=True)
    new_data = new_data[['QUANTITY_FAILS']]
    new_data.rename(columns={'QUANTITY_FAILS':symbol+'_QUANTITY_FAILS'},inplace=True)
    
    return new_data

def read_ftd_data_any_stock(cusip_number,symbol):
    new_df = pd.DataFrame()
    for filename in all_files:
        if filename == 0:
            new_df = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
        else:
            new_df2 = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
            new_df = concat_df(new_df,new_df2)
            
    return new_df

def fix_dataframe(dataframe):
    # For whatever reason, exporting data to CSV file, then reimporting back in using Pandas
    # solves all issues when it comes to reading the data in the column 
    dataframe.to_csv('../FilesTemp/temp.csv')
    dataframe = pd.read_csv(
        Path('../FilesTemp/temp.csv'),
        infer_datetime_format=True,
        parse_dates=True,
        index_col="Date",
    )
    dataframe.sort_index(inplace=True)
    dataframe.drop_duplicates(inplace=True)
    return dataframe

def concat_df(df_1,df_2):
    df = pd.concat([df_1,df_2],axis='rows')
    return df

In [3]:
## IEX Setup and Test 

# Don't seem to need to use real API Key 
# Sandbox API key works just fine for data 

#iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

#token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [4]:
## IEX Calls and Functions 

def get_chart(stock_ticker, chart_range='14m'):
    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df 

def get_close_price(stock_ticker, chart_range='14m'):

    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    df = df[['close']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def get_daily_data(stock_ticker, chart_range='14m'):

    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    df = df[['close','open','low','high','volume']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def join_ftd_and_close(ftd_df,close_df):
    merged_df = ftd_df.merge(close_df, how='inner',right_index=True, left_index=True)
    return merged_df

In [5]:
def do_all(cusip_number,symbol,chart_range='14m'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_close_price(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    return merged_df

def do_more(cusip_number,symbol,chart_range='14m'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_daily_data(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    return merged_df

def do_all_and_export(cusip_number,symbol,chart_range='14m'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_daily_data(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    # Export Data 
    merged_df.to_csv('../FilesExport/'+symbol+'_all_data.csv')
    return merged_df

def export_ftd_csv_data(dataframe,symbol):    
    dataframe.to_csv('../FilesExport/'+symbol+'_ftd_data.csv')

In [6]:
# gme_ftd_df = return_dataframe(CUSIP_number,symbol)
# gme_ftd_df

In [7]:
# gme_chart = get_close_price('GME')
# gme_chart

In [8]:
# combined_df = join_ftd_and_close(gme_ftd_df,gme_chart)
# combined_df

In [9]:
# gme_test = do_all(GME_CUSIP_number,'GME')
# gme_test

In [10]:
# tsla_cusip = "88160R101" 
# tsla_symbol = 'TSLA'

# # tsla_ftd_df = return_dataframe(tsla_cusip,tsla_symbol)
# # tsla_chart = get_close_price(tsla_symbol)
# tsla_ftd_data = do_all(tsla_cusip,tsla_symbol)
# tsla_ftd_data

In [11]:
# aapl_cusip = "037833100" 
# aapl_symbol = 'AAPL'

# # aapl_ftd_df = return_dataframe(aapl_cusip,aapl_symbol)
# # aapl_chart = get_close_price(aapl_symbol)
# aapl_ftd_data = do_all(aapl_cusip,aapl_symbol)
# aapl_ftd_data

In [12]:
# %matplotlib inline
# def plot_fails(data,symbol):
#     plot = data.plot(figsize=(14,5),title=symbol)
#     return plot 

In [13]:
gme_export = do_all_and_export(GME_CUSIP_number,'GME')

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


In [14]:
# AMC , BB , NOK 
amc_cusip = '00165C104'
bb_cusip = '09228F103'
nok_cusip = '654902204'

# TLSA, SPCE, Shopify, QSR 
tsla_cusip = "88160R101" 
shop_cusip = '82509L107'
qsr_cusip = '76131D103'
spce_cusip = '92766K106'

# AAPL, MSFT, JPM, Goldman, and Berkshire 
aapl_cusip = '037833100'
msft_cusip = '594918104'
jpm_cusip = '46625H100'
gs_cusip = '38141G104'

In [15]:
# amc_export = do_all_and_export(amc_cusip,'AMC')
# bb_export = do_all_and_export(bb_cusip,'BB')
# nok_export = do_all_and_export(nok_cusip,'NOK')
# tsla_export = do_all_and_export(tsla_cusip,'TSLA')
# shop_export = do_all_and_export(shop_cusip,'SHOP')
# qsr_export = do_all_and_export(qsr_cusip,'QSR')
# spce_export = do_all_and_export(spce_cusip,'SPCE')
# aapl_export = do_all_and_export(aapl_cusip,'AAPL')
# msft_export = do_all_and_export(msft_cusip,'MSFT')
# jpm_export = do_all_and_export(jpm_cusip,'JPM')
# gs_export = do_all_and_export(gs_cusip,'GS')

In [17]:
## Read FTD file 
data = pd.read_csv(
    Path("../Resources/sec_ftd_1.csv"),
    index_col=False
)
data.head()


,SETTLEMENT DATE|CUSIP|SYMBOL|QUANTITY (FAILS)|DESCRIPTION|PRICE
0,20210816|B38564108|EURN|33389|EURONAV NV ANTWE...
1,20210816|C00948106|AGRI|7719|AGRIFORCE GROWING...
2,20210816|D18190898|DB|66551|DEUTSCHE BANK AG N...
3,20210816|G0R21B112|ACTDW|495|ARCLIGHT CLEAN TR...
4,20210816|G00748114|STWOW|3700|ACON S2 ACQUISIT...


In [21]:
header = "SETTLEMENT DATE|CUSIP|SYMBOL|QUANTITY (FAILS)|DESCRIPTION|PRICE"
data[header] = data[header].str.replace('|',',')
new_data = data.rename(columns={header:'SETTLEMENT_DATE,CUSIP,SYMBOL,QUANTITY_FAILS,DESCRIPTION,PRICE'})
Header = "Header"
new_data = data.rename(columns={header:Header})
new_df = pd.DataFrame()
new_df = new_data.Header.str.split(",",expand=True)
new_df = new_df.rename(columns={0:'Date',1:'CUSIP',2:'SYMBOL',3:'QUANTITY_FAILS',4:'DESCRIPTION',5:'PRICE'})

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [22]:
new_df.head()

,Date,CUSIP,SYMBOL,QUANTITY_FAILS,DESCRIPTION,PRICE,6,7
0,20210816,B38564108,EURN,33389,EURONAV NV ANTWERPEN (BELGIUM),7.75,None,None
1,20210816,C00948106,AGRI,7719,AGRIFORCE GROWING SYS LTD COM,2.89,None,None
2,20210816,D18190898,DB,66551,DEUTSCHE BANK AG NAMEN AKT (DE,12.96,None,None
3,20210816,G0R21B112,ACTDW,495,ARCLIGHT CLEAN TRANSITION CORP,1.12,None,None
4,20210816,G00748114,STWOW,3700,ACON S2 ACQUISITION CORP WT EX,1.39,None,None


In [81]:
cusip_df = new_df[['CUSIP','SYMBOL']]
cusip_df.drop_duplicates(inplace=True)
cusip_df

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CUSIP,SYMBOL
0,B38564108,EURN
1,C00948106,AGRI
2,D18190898,DB
3,G0R21B112,ACTDW
4,G00748114,STWOW
...,...,...
67631,983134107,WYNN
67661,988869103,ZMRK
67664,98907K202,ZNTEU
67678,98978N119,ZIVOW


In [82]:
cusip_df.drop(67689, inplace=True)
cusip_df.reset_index(inplace=True)
cusip_df.drop(columns='index',inplace=True)
cusip_df

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,CUSIP,SYMBOL
0,B38564108,EURN
1,C00948106,AGRI
2,D18190898,DB
3,G0R21B112,ACTDW
4,G00748114,STWOW
...,...,...
12367,98154X105,WSGF
12368,983134107,WYNN
12369,988869103,ZMRK
12370,98907K202,ZNTEU


In [36]:
cusip_list = cusip_df[['CUSIP']]
cusip_list

,CUSIP
0,B38564108
1,C00948106
2,D18190898
3,G0R21B112
4,G00748114
...,...
12367,98154X105
12368,983134107
12369,988869103
12370,98907K202


In [42]:
test_list = cusip_list.head(10)
symb_list = cusip_df[['SYMBOL']]
symb_list = symb_list.head(10)
test_list

test_list_2 = cusip_df.head(10)

In [51]:
cusip_10 = cusip_df.head(10)

In [43]:
# new_stock_list = []
# #symbol_list = 

# for i,x in test_list_2:
#     temp_variable = return_dataframe(i,x)
#     new_stock_list.append(temp_variable)

ValueError: too many values to unpack (expected 2)

In [58]:
# adj = ["red", "big", "tasty"]
# fruits = ["apple", "banana", "cherry"]

# for x in adj:
#   for y in fruits:
#     print(x, y)

# symb_list
# test_list
empty_list = []
# for x in test_list: 
#     for y in symb_list: 
#         temp_var = return_dataframe(x,y)
#         empty_list.append(temp_var)


        
# for key, value in a_dict.items():
# ...     print(key, '->', value)


cusip_10 = cusip_10.head()
# for i in cusip_10.index : 
#     cusip = cusip_10['CUSIP'][i]
#     symbol = cusip_10['SYMBOL'][i]
    
#     empty_list.append(return_dataframe(cusip,symbol))
    
# empty_list

In [59]:
cusip_10

,CUSIP,SYMBOL
0,B38564108,EURN
1,C00948106,AGRI
2,D18190898,DB
3,G0R21B112,ACTDW
4,G00748114,STWOW


In [97]:
#cusip_10['CUSIP'][1]
#C00948106
#test_return = return_dataframe('C00948106',cusip_10['SYMBOL'][2])
#empty_list = []
for ind in cusip_10:
    try:
        temp_var = return_dataframe(cusip_10['CUSIP'][ind],cusip_10['SYMBOL'][ind])
        empty_list.append(temp_var)
    except:
        pass
    
empty_list

[]

In [95]:
# index_var = 15
# test_return = return_dataframe(cusip_df['CUSIP'][index_var],cusip_df['SYMBOL'][index_var])
# test_return

In [76]:
cusip_2 = cusip_df.loc[4]
cusip_2

CUSIP     G00748114
SYMBOL        STWOW
Name: 4, dtype: object

In [103]:
test_return1 = return_dataframe(cusip_10['CUSIP'][0],cusip_10['SYMBOL'][0])
test_return2= return_dataframe(cusip_10['CUSIP'][1],cusip_10['SYMBOL'][1])

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


KeyError: 'C00948106'

In [ ]:
empty1_list = []
empty1_list.append(test_return1)
empty1_list.append(test_return2)
empty1_list